In [1]:
import os
import csv
import textwrap

from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_google_genai import GoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings

# graph 
from yfiles_jupyter_graphs import GraphWidget

from dotenv import load_dotenv

In [2]:
# load the environment
load_dotenv()

# Global constants
VECTOR_INDEX_NAME = "form_10k_chunks"
VECTOR_NODE_LABEL = "Chunk"
VECTOR_SOURCE_PROPERTY = "text"
VECTOR_EMBEDDING_PROPERTY = "textEmbedding"

# initialize the embedding model
GOOGLE_EMBEDDING_MODEL = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
# initiate the google gemini pro model
LLM = GoogleGenerativeAI(model="models/gemini-1.5-pro-latest", temperature=0.0)

In [3]:
# initiate the graph database
# all data are get from .env file for connection with graph database
graph = Neo4jGraph()

## Read the collection of Form 13s
- Investment management firms must report on their investments in companies to SEC by filing a document called **Form 13**
- You'll load a collection of Form 13 for managers that have invested in NetApp
- You can check out the CSV file by navigating to the data directly using the File menu at the top of the notebook

In [4]:
all_form13s = []

with open("./data/form13.csv", mode="r") as csv_file:
    csv_reader = csv.DictReader(csv_file)
    # each row is dictionary
    for row in csv_reader:
        all_form13s.append(row)

print(">>>Total records: ", len(all_form13s))
print(">>>Sample records: ", all_form13s[0])

>>>Total records:  561
>>>Sample records:  {'source': 'https://sec.gov/Archives/edgar/data/1000275/0001140361-23-039575.txt', 'managerCik': '1000275', 'managerAddress': 'ROYAL BANK PLAZA, 200 BAY STREET, TORONTO, A6, M5J2J5', 'managerName': 'Royal Bank of Canada', 'reportCalendarOrQuarter': '2023-06-30', 'cusip6': '64110D', 'cusip': '64110D104', 'companyName': 'NETAPP INC', 'value': '64395000000.0', 'shares': '842850'}


## Create company nodes in the graph
- Use the companies identified in the Form 13s to create `Company` nodes
- Here there is only one company - NetApp

In [5]:
# Select the first row from the form 13s
first_form13s = all_form13s[0]

# Define Cypher query
cypher_query = """
MERGE (com:Company {cusip6: $cusip6})
    ON CREATE
        SET com.companyName = $companyName,
            com.cusip = $cusip
"""

# Execute the query
graph.query(cypher_query, params={
    "cusip6": first_form13s["cusip6"],
    "companyName": first_form13s["companyName"],
    "cusip": first_form13s["cusip"],
})


[]

In [6]:
# validate the query above
cypher = """
MATCH (com:Company)
RETURN com LIMIT 1
"""

graph.query(cypher)

[{'com': {'cusip': '64110D104',
   'companyName': 'NETAPP INC',
   'cusip6': '64110D'}}]

### Update the company name to match Form 10-k

In [7]:
cypher = """
MATCH (com:Company), (form:Form)
    WHERE com.cusip6 = form.cusip6
RETURN com.companyName, form.names
"""
graph.query(cypher)

[{'com.companyName': 'NETAPP INC', 'form.names': ['Netapp Inc', 'NETAPP INC']}]

In [8]:
cypher = """
MATCH (com:Company), (form:Form)
    WHERE com.cusip6 = form.cusip6
SET com.names=form.names
"""

graph.query(cypher)

[]

### Create a `FILED` relationship between the company and the Form-10k node

In [9]:
graph.query("""
MATCH (com:Company), (form:Form)
    WHERE com.cusip6 = form.cusip6
MERGE (com)-[:FILED]->(form)
""")

[]

### Create manaager nodes
- Create a `manager` node for companies that have filed a Form 13 to report their investment in NetApp
- Start with the single manager who filed the Form 13 in the list

In [10]:
cypher = """
MERGE (mgr:Manager {managerCik: $managerParam.managerCik})
  ON CREATE
      SET mgr.managerName = $managerParam.managerName,
          mgr.managerAddress = $managerParam.managerAddress
"""

graph.query(cypher, params={"managerParam": first_form13s})

[]

In [11]:
graph.query("""
MATCH (mgr:Manager)
RETURN mgr LIMIT 1
""")

[{'mgr': {'managerCik': '1000275',
   'managerAddress': 'ROYAL BANK PLAZA, 200 BAY STREET, TORONTO, A6, M5J2J5',
   'managerName': 'Royal Bank of Canada'}}]

### Create a uniqueness constraint to avoid duplicate managers

In [12]:
graph.query("""
CREATE CONSTRAINT unique_manager
    IF NOT EXISTS
    FOR (n:Manager)
    REQUIRE n.managerCik IS UNIQUE
""")

[]

### Create a fulltext index of manager names to enable text search

In [13]:
graph.query("""
CREATE FULLTEXT INDEX fullTextManagerNames
    IF NOT EXISTS
    FOR (mgr:Manager)
    ON EACH [mgr.managerName]
""")


# do search in index as test
results = graph.query("""
CALL db.index.fulltext.queryNodes("fullTextManagerNames",
    "royal bank") YIELD node, score
""")

print(results[0])

{'node': {'managerCik': '1000275', 'managerAddress': 'ROYAL BANK PLAZA, 200 BAY STREET, TORONTO, A6, M5J2J5', 'managerName': 'Royal Bank of Canada'}, 'score': 0.2615291476249695}


### Create nodes for all companies that filed a Form 13

In [14]:
# Define Cypher query
cypher = """
MERGE (mgr:Manager {managerCik: $managerParam.managerCik})
  ON CREATE
      SET mgr.managerName = $managerParam.managerName,
          mgr.managerAddress = $managerParam.managerAddress
"""

# Loop through all Form 13s
for form13 in all_form13s:
    # Execute the query
    graph.query(cypher, params={"managerParam": form13})


In [15]:
# count number of manager in graph
graph.query("""
MATCH (mgr:Manager)
RETURN count(mgr)
""")

[{'count(mgr)': 561}]

## Create relationships between managerrs and companies
- Match companies with managers based on data in the Form 13
- Create an `OWNS_STOCK_IN` relationship between the manager and the company
- Dtart with the single manager who filed the first Form 13 in the list

In [16]:
cypher = """
MATCH (mgr:Manager {managerCik: $investmentParam.managerCik}),
    (com:Company {cusip6: $investmentParam.cusip6})
RETURN mgr.maangerName, com.companyName, $investmentParam as inverstmenet
"""

graph.query(cypher, params={
    "investmentParam": first_form13s
})

[{'mgr.maangerName': None,
  'com.companyName': 'NETAPP INC',
  'inverstmenet': {'shares': '842850',
   'source': 'https://sec.gov/Archives/edgar/data/1000275/0001140361-23-039575.txt',
   'managerName': 'Royal Bank of Canada',
   'managerAddress': 'ROYAL BANK PLAZA, 200 BAY STREET, TORONTO, A6, M5J2J5',
   'value': '64395000000.0',
   'cusip6': '64110D',
   'cusip': '64110D104',
   'reportCalendarOrQuarter': '2023-06-30',
   'companyName': 'NETAPP INC',
   'managerCik': '1000275'}}]

In [17]:
# create relationship between manager and company for first record.
# relation has property
cypher = """
MATCH (mgr:Manager {managerCik: $ownsParam.managerCik}),
    (com:Company {cusip6: $ownsParam.cusip6})
MERGE (mgr)-[owns:OWNS_STOCK_IN {
    reportCalendarOrQuarter: $ownsParam.reportCalendarOrQuarter
}]->(com)
ON CREATE
    SET owns.value = toFloat($ownsParam.value),
        owns.shares = toInteger($ownsParam.shares)
RETURN mgr.managerName, owns.reportCalendarOrQuarter, com.companyName
"""

graph.query(cypher, params={
    "ownsParam": first_form13s})

[{'mgr.managerName': 'Royal Bank of Canada',
  'owns.reportCalendarOrQuarter': '2023-06-30',
  'com.companyName': 'NETAPP INC'}]

In [18]:
cypher = """
MATCH (mgr:Manager {managerCik: $ownsParam.managerCik})
-[owns:OWNS_STOCK_IN]->(com:Company { cusip6: $ownsParam.cusip6})
RETURN owns {.shares, .value }
"""

graph.query(cypher, params={"ownsParam": first_form13s})

[{'owns': {'shares': 842850, 'value': 64395000000.0}}]

### Create relationships between all of the maangers who filed Form 13s and the company

In [19]:
cypher = """
MATCH (mgr:Manager {managerCik: $ownsParam.managerCik}),
    (com:Company {cusip6: $ownsParam.cusip6})
MERGE (mgr)-[owns:OWNS_STOCK_IN {
    reportCalendarOrQuarter: $ownsParam.reportCalendarOrQuarter
    }]->(com)
    ON CREATE 
        SET owns.value = toFloat($ownsParam.value),
            owns.shares = toInteger($ownsParam.shares)
"""

# loop through all Form 13s
for form13 in all_form13s:
    graph.query(cypher, params={"ownsParam": form13})

# display how much is reeleation of manager owning stock in company
cypher = """
MATCH (:Manager)-[owns:OWNS_STOCK_IN]->(:Company)
RETURN count(owns) as investments
"""

graph.query(cypher)

[{'investments': 561}]

### Determine the number of investors
- Start by finding a form 10-k chunk, and save to use in subsequent queries

In [20]:
cypher = """
MATCH (chunk:Chunk)
RETURN chunk.chunkId as ChunkId LIMIT 1
"""

chunk_rows = graph.query(cypher)
print(chunk_rows)

[{'ChunkId': '0000950170-23-027948--item1-chunk0000'}]


### Display shcema 

In [21]:
graph.refresh_schema()
print(textwrap.fill(graph.schema, 80))

Node properties: Movie {title: STRING, taglineEmbedding: LIST, tagline: STRING,
released: INTEGER} Person {born: INTEGER, name: STRING} Chunk {text: STRING,
textEmbedding: LIST, source: STRING, f10kItem: STRING, chunkSeqId: INTEGER, cik:
STRING, cusip6: STRING, chunkId: STRING, names: LIST, formId: STRING} Form {cik:
STRING, cusip6: STRING, names: LIST, formId: STRING, source: STRING} Company
{cusip6: STRING, names: LIST, companyName: STRING, cusip: STRING} Manager
{managerName: STRING, managerCik: STRING, managerAddress: STRING} Relationship
properties: SECTION {f10kItem: STRING} OWNS_STOCK_IN {shares: INTEGER,
reportCalendarOrQuarter: STRING, value: FLOAT} The relationships:
(:Person)-[:WORKS_WITH]->(:Person) (:Chunk)-[:NEXT]->(:Chunk)
(:Chunk)-[:PART_OF]->(:Form) (:Form)-[:SECTION]->(:Chunk)
(:Company)-[:FILED]->(:Form) (:Manager)-[:OWNS_STOCK_IN]->(:Company)


In [22]:
from neo4j import GraphDatabase

# directly show the graph resulting from the given Cypher query
default_cypher = "MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t LIMIT 50"

def showGraph(cypher: str = default_cypher):
    # create a neo4j session to run queries
    driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))
    session = driver.session()
    widget = GraphWidget(graph = session.run(cypher).graph())
    widget.node_label_mapping = 'id'
    #display(widget)
    return widget

showGraph()

GraphWidget(layout=Layout(height='790px', width='100%'))

### Determine the number of investors
- Start by finding a form 10-k chunk and save to use in subsequent queries 

In [23]:
cypher = """
MATCH (chunk:Chunk)
RETURN chunk.chunkId as chunkId LIMIT 1
"""

chunk_rows = graph.query(cypher)
chunk_rows

[{'chunkId': '0000950170-23-027948--item1-chunk0000'}]

In [24]:
ref_chunk_id = chunk_rows[0]["chunkId"]

cypher = """
MATCH (:Chunk {chunkId: $chunkIdParam})-[:PART_OF]->(f:Form)
RETURN f.source
"""

graph.query(cypher, params={"chunkIdParam": ref_chunk_id})

[{'f.source': 'https://www.sec.gov/Archives/edgar/data/1002047/000095017023027948/0000950170-23-027948-index.htm'}]

In [25]:
cypher = """
MATCH (:Chunk {chunkId: $chunkIdParam})-[:PART_OF]->(f:Form),
    (com:Company)-[:FILED]->(f)
RETURN com.companyName as name
"""

# execute the query
graph.query(cypher, params={'chunkIdParam': ref_chunk_id})

[{'name': 'NETAPP INC'}]

In [26]:
cypher_query = """
MATCH (:Chunk {chunkId: $chunkIdParam})-[:PART_OF]->(f:Form),
(com:Company)-[:FILED]->(f),
(mgr:Manager)-[:OWNS_STOCK_IN]->(com)
RETURN com.companyName,
count(mgr.managerName) as numberOfInvestors
LIMIT 1
"""

result = graph.query(cypher_query, params={
    "chunkIdParam": ref_chunk_id
})

print(result)

[{'com.companyName': 'NETAPP INC', 'numberOfInvestors': 561}]


### Use queries to build additional context for LLM
- Create sentences that indicate how much stock a manager has invested in a company

In [27]:
cypher = """
MATCH (:Chunk {chunkId: $chunkIdParam})-[:PART_OF]->(f:Form),
    (com:Company)-[:FILED]->(f),
    (mgr:Manager)-[owns:OWNS_STOCK_IN]->(com)
RETURN mgr.managerName + " owns " + owns.shares +
    " shares of " + com.companyName + 
    " at a value of $" + 
    apoc.number.format(toInteger(owns.value)) AS text
LIMIT 10
"""

results = graph.query(cypher, params={
    "chunkIdParam": ref_chunk_id
})

results

[{'text': 'CSS LLC/IL owns 12500 shares of NETAPP INC at a value of $955,000,000'},
 {'text': 'BOKF, NA owns 40774 shares of NETAPP INC at a value of $3,115,134,000'},
 {'text': 'BANK OF NOVA SCOTIA owns 18676 shares of NETAPP INC at a value of $1,426,847,000'},
 {'text': 'Jefferies Financial Group Inc. owns 23200 shares of NETAPP INC at a value of $1,772,480,000'},
 {'text': 'DEUTSCHE BANK AG\\ owns 929854 shares of NETAPP INC at a value of $71,040,845,000'},
 {'text': 'TORONTO DOMINION BANK owns 183163 shares of NETAPP INC at a value of $13,984,000'},
 {'text': 'STATE BOARD OF ADMINISTRATION OF FLORIDA RETIREMENT SYSTEM owns 265756 shares of NETAPP INC at a value of $20,303,759,000'},
 {'text': 'NISA INVESTMENT ADVISORS, LLC owns 67848 shares of NETAPP INC at a value of $5,183,587,000'},
 {'text': 'ONTARIO TEACHERS PENSION PLAN BOARD owns 7290 shares of NETAPP INC at a value of $556,956,000'},
 {'text': 'STATE STREET CORP owns 9321206 shares of NETAPP INC at a value of $712,140,138,0

In [28]:
results[0]["text"]

'CSS LLC/IL owns 12500 shares of NETAPP INC at a value of $955,000,000'

- Create a plain Question Answer chain
- Similarity search only, no augmentation by Cypher Query

In [29]:
vector_store = Neo4jVector.from_existing_graph(
    embedding=GOOGLE_EMBEDDING_MODEL,
    index_name=VECTOR_INDEX_NAME,
    node_label=VECTOR_NODE_LABEL,
    text_node_properties=[VECTOR_SOURCE_PROPERTY],
    embedding_node_property=VECTOR_EMBEDDING_PROPERTY,
)

# create a retriever from the vector store
retriever = vector_store.as_retriever()

# create a chain from retriver
plain_chain = RetrievalQAWithSourcesChain.from_chain_type(
    LLM,
    chain_type="stuff",
    retriever=retriever
)

- Create a second QA chain
- Auugment similarity search using sentences found by investment query above

In [30]:
investment_retrieval_query = """
MATCH (node)-[:PART_OF]->(f:Form),
    (f)<-[:FILED]-(com:Company),
    (com)<-[owns:OWNS_STOCK_IN]-(mgr:Manager)
WITH node, score, mgr, owns, com 
    ORDER BY owns.shares DESC LIMIT 10
WITH collect (
    mgr.managerName + 
    " owns " + owns.shares + 
    " shares in " + com.companyName + 
    " at a value of $" + 
    apoc.number.format(toInteger(owns.value)) + "." 
) AS investment_statements, node, score
RETURN apoc.text.join(investment_statements, "\n") + 
    "\n" + node.text AS text,
    score,
    { 
      source: node.source
    } as metadata
"""

In [31]:
# Create a Neo4j vector store with investment information
# uses both structured data and instructured data while retriving the information
vector_store_with_investment = Neo4jVector.from_existing_index(
    GOOGLE_EMBEDDING_MODEL,
    index_name=VECTOR_INDEX_NAME,
    # text_node_properties=[VECTOR_SOURCE_PROPERTY],
    retrieval_query=investment_retrieval_query,
)

# Create a retriever from the vector store
retriever_with_investments = vector_store_with_investment.as_retriever()

# Create a QA chain with investment sources
investment_chain = RetrievalQAWithSourcesChain.from_chain_type(
    LLM,
    chain_type="stuff",
    retriever=retriever_with_investments,
    verbose=True
)

### Compare the outputs

In [32]:
question = "In a single sentence, tell me about Netapp."

In [33]:
# answer by plain chain
plain_chain.invoke({"question": question})

{'question': 'In a single sentence, tell me about Netapp.',
 'answer': 'FINAL ANSWER: Netapp is a company that provides cloud services and storage infrastructure powered by intelligent data management software.\n',
 'sources': 'https://www.sec.gov/Archives/edgar/data/1002047/000095017023027948/0000950170-23-027948-index.htm'}

In [34]:
# answer by investment chain
investment_chain.invoke({"question": question})



> Entering new RetrievalQAWithSourcesChain chain...

> Finished chain.


{'question': 'In a single sentence, tell me about Netapp.',
 'answer': 'NetApp is a cloud-led, data-centric software company that helps organizations manage data across hybrid multicloud environments. \n',
 'sources': 'https://www.sec.gov/Archives/edgar/data/1002047/000095017023027948/0000950170-23-027948-index.htm'}

In [35]:
question = "In a single sentence, tell me about Netapp investors."

# lets see about investors in Netapp
plain_answer = plain_chain.invoke({"question": question})
investment_chain_answer = investment_chain.invoke({"question": question})

print("PLAIN_ANSWER: ", plain_answer)
print("\n\nANSWER_BY_INVESTMENT_CHAIN: ", investment_chain_answer)



> Entering new RetrievalQAWithSourcesChain chain...

> Finished chain.
PLAIN_ANSWER:  {'question': 'In a single sentence, tell me about Netapp investors.', 'answer': 'NetApp investors are individuals or institutions who hold a financial stake in the company, which specializes in cloud data services and data management solutions. \n', 'sources': 'https://www.sec.gov/Archives/edgar/data/1002047/000095017023027948/0000950170-23-027948-index.htm'}


ANSWER_BY_INVESTMENT_CHAIN:  {'question': 'In a single sentence, tell me about Netapp investors.', 'answer': "FINAL ANSWER: NetApp's major investors include large investment firms like Vanguard Group Inc., BlackRock Inc., and Primecap Management Co. \n", 'sources': 'https://www.sec.gov/Archives/edgar/data/1002047/000095017023027948/0000950170-23-027948-index.htm'}
